<div style="text-align:center">Marco Seiz, Andreas Prahs, Anastasia August<br>
KIT, Institut für Angewandte Materialien (IAM)<br>
Mikrostrukturmodellierung und Simulation (MMS)
</div>
<h1>Mikrostruktursimulation <br/>Zusatzbeispiel zur numerischen Lösung von PDEs </h1>

Advektionsgleichung: Ein Größe $c$ wird mit der Geschwindigkeit $v$ durch das Gebiet bewegt.
$$ \frac{\partial c}{\partial t} = -v \frac{\partial c}{\partial x} $$

Die räumliche Ableitung wird hier nicht mit zentralen Differenzen gebildet, da das Schema unbedingt instabil ist, sondern mit rechtsseitigen Differenzen für $v<0$ und mit linksseitigen für $v>0$, hier für $v<0$ exemplarisch:

$$ \frac{\partial c}{\partial t} = -v \frac{c_{i+1}-c_i}{\Delta x} $$

und die zeitliche Ableitung wird wieder mit dem expliziten Eulerverfahren bestimmt und umgestellt:

$$ c_i^{n+1} = c_i^n - \Delta t v \frac{c_{i+1}^n-c_i^n}{\Delta x}. $$

Vgl. hierzu auch http://bender.astro.sunysb.edu/classes/phy688_spring2013/lectures/advection.pdf . Die richtige numerische Lösung der Advektionsgleichung ist ein Gebiet für sich. 

Es werden beispielhaft weitere Terme auf der rechten Seite der Gleichung eingeführt, um daraus jeweils eine 

a) Advektions-Diffusions-Gleichung $$ \frac{\partial c}{\partial t} = -v \frac{\partial c}{\partial x} + D \frac{\partial^2 c}{\partial x^2} $$

b) Advektions-Diffusions-Reaktions-Gleichung $$ \frac{\partial c}{\partial t} = -v \frac{\partial c}{\partial x} + D \frac{\partial^2 c}{\partial x^2} - Ac(1-c)(1-2c) $$

zu erhalten. Der diffusive Anteil wird wie in der Übung diskretisiert, während der Reaktionsanteil ohne eine Diskretisierung auskommt, da in diesem keine Ableitungen vorkommen.

Diese Terme können einzeln über das passende Setzen ihrer Vorfaktoren ($v$, $D$, $A$) an- und ausgeschaltet werden.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
mpl.rcParams.update({'font.size': 16})
mpl.rcParams['figure.figsize'] = (9,6)
mpl.rcParams['axes.grid'] = True
from numba import jit
import scipy.signal

In [ ]:
v=1 #Geschwindigkeit (- -> nach links, + -> nach rechts)
D=0 # Diffusivität > 0
A=0 # Vorfaktor für reaktionsterm: 0 oder 1 zum Aus-/Anschalten
# NB: der Reaktionsterm ergibt ohne Diffusionsterm keine sinnvollen Ergebnisse
vabs = abs(v)
deltax=1
# da ein explizites Verfahren verwendet wird, gibt es maximal stabile Zeitschrittweiten
deltat =  1*min( deltax/max(vabs, 1e-16), deltax**2 / (2*2*max(D, 1e-16)))
offset1 = int(v < 0) # konstantes upwind/downwind-verfahren
offset2 = int(v > 0)

In [ ]:
C = vabs * deltat / deltax # Courantzahl
# bei der reinen Advektionsgleichung:
# für C < 1 gibt es eine numerischen Diffusion des Profils
# bei C > 1 numerische Instabilität
# bei C = 1 wird das Profil erhalten
C

In [ ]:
@jit
def pde(field, fieldNew):
    length = field.shape[0]
    for i in range(1,length-1): 
        fieldNew[i] = field[i] + deltat *  ( -v * (field[i+offset1] - field[i-offset2]) / deltax 
                                           + D * ( field[i+1] - 2*field[i] + field[i-1] ) / (deltax * deltax)   
                                           - A*field[i] * (1-field[i]) * (1-2*field[i]))
                                            
def ini(field): # "top hat"-Profil
    mylen = gridpoints//4
    field[mylen:3*mylen] = 1
        
def iniGauss(field): # Gaußsche Glocke
    field[:] = scipy.signal.gaussian(gridpoints, 10, True)
    
def bcs(field): # periodisch
    field[0] = field[-2]
    field[-1] = field[1] 

In [ ]:
s = ""
if v != 0:
    s+="Advektions-"
if D != 0:
    s+= "Diffusions-"
if A != 0:
    s+= "Reaktions-"
print("Es wird eine %sGleichung gelöst." % (s))

In [ ]:
# Initialisieren der Felder
gridpoints=100
cold=np.zeros(gridpoints)
cnew=np.zeros(gridpoints)
iniGauss(cold)
#ini(cold)
framelist_e=[]
times_e=[]

In [ ]:
#Simulation
goaltime = 98
# goaltime = 98 ~ eine Advektionsbewegung durchs Gebiet (letztes Profil == erstes Profil für reine Advektion und C=1)
# goaltime = 30 ~ man erkennt, was der Diffusions-Reaktionsanteil bewirkt
print("Zielzeit ist %lf" % (goaltime))
timesteps=int(goaltime/deltat)
writefreq=int(timesteps/5) 
bcs(cold)
t=0
#start = time.time()
for i in range(timesteps):
    if i%writefreq == 0:
        framelist_e.append(cold.copy())
        times_e.append(t)
    pde(cold, cnew)
    bcs(cnew)
    cold, cnew = cnew, cold
    t+=deltat
#end = time.time()
if i % writefreq != 0:
    framelist_e.append(cold.copy())
    times_e.append(i*deltat)
#framelist_e.append(cold.copy())
#times_e.append(i*deltat)
#print("Laufzeit: %f Sekunden" % (end-start))

In [ ]:
fig_e, ax_e = plt.subplots()
for i in range(len(framelist_e)):
    ax_e.plot(framelist_e[i], label="%.1f" % (times_e[i]))
    
ax_e.set_xlabel("Gitterposition / -")
ax_e.set_ylabel("Feldwert / -")

ax_e.legend(ncol=2,title="Simulationszeit")